<a href="https://colab.research.google.com/github/shouvikcirca/LLMs/blob/openai/langchain_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! python --version
! pip install --upgrade langchain
! pip install python-dotenv
! pip install --upgrade openai
! pip install pypdf
! pip install tiktoken
! pip install chromadb
# ! pip install cohere


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
# ! ls drive/MyDrive

In [3]:
from langchain.document_loaders import PyPDFLoader
from dotenv import load_dotenv, find_dotenv
import os
import sys
import openai

In [4]:
a = load_dotenv('drive/MyDrive/env')
openai.api_key = os.environ['OPENAI_API_KEY']


In [5]:
loader = PyPDFLoader('drive/MyDrive/C-CARES_FRS_Grievance_Redressal.pdf')
pages = loader.load()
len(pages)

17

In [ ]:
# page = pages[3]
# print(page.page_content[:1000])

In [ ]:
# page.metadata

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, TokenTextSplitter

In [7]:
chunk_size = 50
chunk_overlap = 10

In [8]:
# r_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=chunk_size,
#     chunk_overlap=chunk_overlap
# )
# c_splitter = CharacterTextSplitter(
#     chunk_size=chunk_size,
#     chunk_overlap=chunk_overlap,
#     length_function = len
# )
t_splitter = TokenTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

In [9]:
docs = t_splitter.split_documents(pages)
len(docs)

160

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(
    api_key=openai.api_key
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [11]:
from langchain.vectorstores import Chroma

In [ ]:
# persist_directory = 'docs/chroma/'

In [13]:
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    # persist_directory=persist_directory
)

In [14]:
print(vectordb._collection.count())

160


In [15]:
question = "What is the grievance redressal module designed for ?"

In [16]:
similardocs = vectordb.similarity_search(question, k = 5)

In [17]:
# vectordb.persist() # So that we can use the vector database in the future

In [19]:
! pip install -qU langchain-openai

In [20]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=openai.api_key)

In [21]:
from langchain.chains import RetrievalQA

In [22]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Elaborate as much as possible.
{context}. Don not append anything that is not relevant ot the question. Answer in the following format

Here is the question
{question}

Answer in the following format
Answer: Answer


"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [25]:
from langchain_openai import OpenAI

llm = OpenAI(api_key=openai.api_key)

In [26]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [27]:
result = qa_chain({"query": "How many functional requirements are there ?"})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [28]:
result["result"]

'Answer: There are four functional requirements for the Grievance Redressal system: Grievance Category, Grievance Description, Supporting Documents, and File Upload.'